In [1]:
import pandas as pd
import geopandas as gpd

In [34]:
dtypes = {
    'ELEGIBLE_VOTERS' : int,
    'TOT_VOTES' :     int,
    'PART_PERCENT' :  float,
    'VALID_VOTES' :   'int',
    'YES' :           'int64',
    'NO' :            'int64',
    'YES_IN_PER' :    'float64',
    'KTNR' :    'float64'
}

In [35]:
df  = gpd.read_file('master_w_bikes.geojson', driver='GeoJSON', dtype=dtypes)

In [103]:
def read_population():
    results = pd.read_excel('/Users/Dario/Documents/GitHub/GIS_KARTO/data/bevoelkerung/bev.xlsx',
                            sheet_name='data', header=3)
    results = results.dropna(axis=0)

    df = results.loc[~results['GMDNAME'].str.startswith('>>')]
    df = df.loc[~df['GMDNAME'].str.startswith('-')]
    df['GMDNAME'] = df['GMDNAME'].str[6:]
    df['GMDNAME'] = df['GMDNAME'].str[:4]
    df = df.drop(index=0)
    df = df.reset_index(drop=True)
    df['BFS'] = df['GMDNAME'].astype(int)

    return df

In [104]:
population = read_population()

In [105]:
rename_cols = {
    'BFS':'BFSNR',
    'Total':'POP_TOTAL',
    'Total.1':'POP_CH_TOTAL',
    'Total.2':'POP_FOREIGN_TOTAL',
}

drop_cols = ['GMDNAME','Mann', 'Frau', 'Mann.1', 'Frau.1', 'Mann.2', 'Frau.2']

In [106]:
population = population.rename(columns=rename_cols)
population = population.drop(columns=drop_cols)
population = population.reindex(columns=['BFSNR', 'POP_TOTAL', 'POP_CH_TOTAL', 'POP_FOREIGN_TOTAL'])

In [108]:
df = df.merge(population, on='BFSNR', how='left')

In [110]:
df.to_file('master_table.geojson', driver='GeoJSON')